# Prep data

This is unnecessary to run if you have the "case_study.csv" dataset locally.

In [1]:
# import pandas as pd
# from sublimpy import tidy, utils

In [2]:
# start_date = '20221130'
# end_date = '20230509'

# # open files
# tidy_df = pd.read_parquet(f'../paper1/tidy_df_{start_date}_{end_date}_noplanar_fit.parquet')

# # convert time column to datetime
# tidy_df['time'] = pd.to_datetime(tidy_df['time'])
# tidy_df = utils.modify_df_timezone(tidy_df, 'UTC', 'US/Mountain')

# # limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
# tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:end_date].reset_index()

# ## Add combined blowing snow flux variable
# tidy_df = tidy.tidy_df_add_variable(
#     tidy_df,
#     (
#         tidy_df.query("variable == 'SF_avg_1m_ue'")['value'].values + 
#         tidy_df.query("variable == 'SF_avg_2m_ue'")['value'].values
#     ), 
#     'SF_avg_ue',
#     'snow flux',
#     1,
#     'ue',
# )

In [3]:
# case_src = tidy_df[(tidy_df.time >= '4/15/2023') & (tidy_df.time < '4/20/2023')]
# case_src = case_src[case_src.measurement.isin(['temperature', 'surface temperature', 'relative humidity', 'specific humidity', 'relative humidity', 'w_h2o_', 'snow flux', 'snow depth'])]
# case_src = case_src[~ case_src.variable.str.contains("predicted")]

In [4]:
# case_src_wide = case_src.pivot_table(
#     index = 'time',
#     columns='variable',
#     values='value'
# ).reset_index()

In [5]:
# case_src.to_csv("case_study_tidy.csv", index=False)

In [6]:
# case_src_wide.to_csv("case_study_wide.csv", index=False)

# Run analysis

Read in "case_study.csv" and analyze the data

In [7]:
import altair as alt
import pandas as pd
alt.data_transformers.enable('json')
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

In [8]:
case_src = pd.read_csv("case_study_tidy.csv")
case_src.time = pd.to_datetime(case_src.time)
case_src.head()

,time,variable,value,height,tower,measurement
0,2023-04-15,w_h2o__1m_ue,0.000000,1.0,ue,w_h2o_
1,2023-04-15,T_18m_c,-5.653895,18.0,c,temperature
2,2023-04-15,w_h2o__1m_d,0.000000,1.0,d,w_h2o_
3,2023-04-15,specifichumidity_6m_c,0.003116,6.0,c,specific humidity
4,2023-04-15,w_h2o__5m_c,0.001834,5.0,c,w_h2o_


In [9]:
bs_flux_and_snow_depth_chart = (
    alt.Chart(
        case_src.query("variable == 'SF_avg_ue'")
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q").scale(type='symlog').title(
            "Blowing snow flux (g/m^2/s)"
        )
    ).properties(height = 200) 
    + 
    alt.Chart(
        case_src.query("variable == 'SnowDepth_c'")
    ).mark_line().encode(
        alt.X("time:T"),
        alt.Y("value:Q").scale(type='symlog').title(
            "Snow depth (m)"
        )
    ).properties(height = 200)
).resolve_scale(y='independent')

In [10]:
hl_flux_chart = alt.Chart(
    case_src[
        case_src.time.dt.minute == 0
    ][
        case_src.time.dt.hour % 2 == 0
    ].query(
        "measurement == 'w_h2o_'"
    ).query(
        "tower == 'c'"
    ).query("height > 1")
).mark_line().encode(
    alt.X("value:Q").sort('-y').title("<w'q'> (g/m²/s)"),
    alt.Y("height:Q").title("Height (m)"),
    alt.Color("hours(time):O").scale(scheme='rainbow'),
    alt.Facet("date(time):T")
).properties(
    width=200, 
    height = 200
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_42927/1771112374.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  case_src[


In [11]:
temp_chart = alt.Chart(
    case_src[
        case_src.time.dt.minute == 0
    ][
        case_src.time.dt.hour % 2 == 0
    ][
        case_src.measurement.isin(['temperature', 'surface temperature'])
    ].query("tower == 'c'")
).mark_line().encode(
    alt.X("value:Q").sort('-y').title("Temperature (˚C)"),
    alt.Y("height:Q").title("Height (m)"),
    alt.Color("hours(time):O").scale(scheme='rainbow'),
    alt.Facet("date(time):T")
).properties(
    width=200, 
    height = 200
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_42927/2652316108.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  case_src[
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_42927/2652316108.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  case_src[


In [12]:
spechumidity_chart = alt.Chart(
    case_src[
        case_src.time.dt.minute == 0
    ][
        case_src.time.dt.hour % 2 == 0
    ].query(
        "measurement == 'specific humidity'"
    ).query(
        "tower == 'c'"
    )
).mark_line().encode(
    alt.X("value:Q").sort('-y').title("Specific humidity (g/g)"),
    alt.Y("height:Q").title("Height (m)"),
    alt.Color("hours(time):O").scale(scheme='rainbow'),
    alt.Facet("date(time):T")
).properties(
    width=200, 
    height = 200
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_42927/1741176887.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  case_src[


In [13]:
combo_chart = (
    bs_flux_and_snow_depth_chart & hl_flux_chart & temp_chart & spechumidity_chart
).resolve_scale(
    color='independent'
)
combo_chart.save("chriscox_analysis_20240709.png", dpi=300)
combo_chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
